## Architecture-Specific Head Designs (v3)

### Design Principles

1. **GlobalAveragePooling2D for all backbones**
   - Better for spatially-aggregated GEI motion patterns
   - More robust to position variations
   - Smoother gradients → better convergence

2. **2-layer heads**
   - More capacity than 1-layer universal head
   - Less prone to overfitting than 3-layer complex heads

3. **Activation matching**
   - EfficientNet: `swish` (native to EfficientNet design)
   - ResNet/VGG/MobileNet: `relu` (standard, proven activation)

4. **Progressive dropout**
   - Higher dropout in first Dense layer
   - Lower dropout in second Dense layer
   - Prevents overfitting while maintaining capacity

5. **Size scaling**
   - Larger heads for deeper backbones (ResNet50: 1024→512)
   - Smaller heads for efficient architectures (MobileNet: 256→128)

---

### Head Architectures

**EfficientNet Family (B0, B2, B3):**
```
GlobalAveragePooling2D
    ↓
Dense(512, activation='swish')
    ↓
BatchNormalization
    ↓
Dropout(0.3)
    ↓
Dense(256, activation='swish')
    ↓
Dropout(0.2)
    ↓
Dense(num_classes, activation='softmax')
```
- **Rationale:** EfficientNet uses swish activation internally, BatchNorm for stability
- **Capacity:** 512→256 sufficient for moderate-sized backbone

**ResNet50:**
```
GlobalAveragePooling2D
    ↓
Dense(1024, activation='relu')
    ↓
Dropout(0.4)
    ↓
Dense(512, activation='relu')
    ↓
Dropout(0.3)
    ↓
Dense(num_classes, activation='softmax')
```
- **Rationale:** ResNet50 is deep (50 layers) → larger head capacity needed
- **Capacity:** 1024→512 to match backbone complexity

**VGG16:**
```
GlobalAveragePooling2D
    ↓
Dense(512, activation='relu')
    ↓
Dropout(0.5)
    ↓
Dense(256, activation='relu')
    ↓
Dropout(0.4)
    ↓
Dense(num_classes, activation='softmax')
```
- **Rationale:** VGG16 has simple architecture but many parameters → higher dropout
- **Capacity:** 512→256 balanced for VGG's feature space

**MobileNet Family (V2, V3-Large):**
```
GlobalAveragePooling2D
    ↓
Dense(256, activation='relu')
    ↓
Dropout(0.25)
    ↓
Dense(128, activation='relu')
    ↓
Dropout(0.15)
    ↓
Dense(num_classes, activation='softmax')
```
- **Rationale:** MobileNet is lightweight → smaller head maintains efficiency
- **Capacity:** 256→128 matched to compact backbone

---

## Training Strategy (3-Way Split with Validation Monitoring)

### Data Split (Subject-Independent)
- **Training set (~55% of subjects)**: Used for model learning
- **Validation set (~15% of subjects)**: Used for early stopping and learning rate scheduling
- **Test set (~30% of subjects)**: Used ONLY for final evaluation (never seen during training)

✅ **No Data Leakage:** All samples from a subject stay in the same group (train/val/test)

**Example:** If "Volunteer_18" is in the test set → ALL their samples (front view, side view, all exercises) remain in test set

### Phase 1: Frozen Backbone (up to 50 epochs)
- All backbone layers frozen
- Train only architecture-specific classification head
- Learning rate: 0.001
- **Callbacks:** EarlyStopping (patience=10), ReduceLROnPlateau, ModelCheckpoint
- **Monitoring:** Validation loss (stops when validation stops improving)

### Phase 2: Full Fine-tuning (up to 100 epochs)
- Unfreeze ALL layers (simple 100% unfreezing)
- Learning rate: 0.0001 (10× reduction)
- **Same callbacks** with validation monitoring
- Allows backbone to adapt to GEI patterns

### Data Augmentation (Basic)
- Horizontal flip: `True`
- Translation: ±15% (height + width)
- **No rotation/zoom/brightness** (keep it simple)

### Why This Approach?

**Problem with no validation set:**
- Model might overfit after epoch 30, but keeps training to epoch 60
- Final weights could be worse than intermediate checkpoints
- No way to know when to stop training

**With validation monitoring:**
- EarlyStopping saves best weights when validation loss stops improving
- ReduceLROnPlateau reduces learning rate when stuck
- Test accuracy becomes a TRUE unseen benchmark

### Evaluation Protocol
- **5 runs per backbone** (quick validation phase)
- Subject-independent 3-way split (55%/15%/30%)
- Test set evaluated ONLY at the end (after all training completes)
- Metrics: Training accuracy, validation accuracy, test accuracy, confusion matrix
- **Success criteria (mean test accuracy across 5 runs):**
  - Minimum: At least one backbone achieves **mean > 84%** (beats baseline 83.25%)
  - Good: Best backbone achieves **mean > 86%** with **std < 2.5%**
  - Excellent: Best backbone achieves **mean > 88%** with **std < 2%**

---

## Setup: TensorFlow Configuration

This cell configures TensorFlow to suppress warnings and enable GPU memory growth.

**Key configurations:**
- Suppress TensorFlow/CUDA warnings for cleaner output
- Enable GPU memory growth (prevents out-of-memory errors)
- Set logging levels to ERROR only

In [ ]:
# CRITICAL: Run this cell FIRST before any other imports
# Suppress TensorFlow warnings at the OS level before TensorFlow loads
import os
import sys
import warnings
import io
import tensorflow as tf

# Set environment variables BEFORE TensorFlow is imported anywhere
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0=all, 1=filter INFO, 2=filter WARNING, 3=errors only
os.environ['AUTOGRAPH_VERBOSITY'] = '0'   # Disable AutoGraph conversion warnings

# Filter Python warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Suppress absl logging (used by TensorFlow internally)
try:
    from absl import logging as absl_logging
    absl_logging.set_verbosity(absl_logging.ERROR)
except ImportError:
    pass

# Redirect stderr temporarily to suppress any remaining warnings during TF import
stderr_backup = sys.stderr
sys.stderr = io.StringIO()

# Restore stderr
sys.stderr = stderr_backup

# Final TensorFlow logging configuration
try:
    tf.get_logger().setLevel('ERROR')
    tf.autograph.set_verbosity(0)
except Exception:
    pass

# Enable GPU memory growth
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
except Exception:
    pass

print("   TF_CPP_MIN_LOG_LEVEL:", os.environ.get('TF_CPP_MIN_LOG_LEVEL'))
print("   AUTOGRAPH_VERBOSITY:", os.environ.get('AUTOGRAPH_VERBOSITY'))
print("   TensorFlow version:", tf.__version__)
print("   GPUs detected:", len(tf.config.list_physical_devices('GPU')))

✅ TensorFlow imported with all warnings suppressed
   TF_CPP_MIN_LOG_LEVEL: 3
   AUTOGRAPH_VERBOSITY: 0
   TensorFlow version: 2.10.0
   GPUs detected: 1


## Import Modules

**Key imports for Experiment 3:**
- `build_model_for_backbone_v3` - New refined architecture-specific heads
- `train_one_run` from `experiment_3` - Training script with v3 heads
- Same data loading and utility functions as Experiment 1

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import logging
import yaml
from tqdm import tqdm
from pathlib import Path
import sys

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Import project modules
from src.data import load_front_side_geis
from src.scripts.experiment_4 import train_experiment_4
from src.utils.io_utils import load_config
from src.utils import (
    set_global_seed,
    setup_results_folder_for_backbone,
    save_experiment_summary,
    get_all_model_parameters,
    load_backbone_results_with_config,
    create_comprehensive_comparison,
    generate_statistical_comparison,
)

root_logger = logging.getLogger()
if root_logger.hasHandlers():
    for handler in root_logger.handlers[:]:
        root_logger.removeHandler(handler)

logging.basicConfig(level=logging.WARNING, format='%(message)s')
logger = logging.getLogger(__name__)

print("✅ All modules imported successfully")

✅ All modules imported successfully


In [ ]:
# Load YAML configuration
config_path = project_root / 'config' / 'experiment_4.yaml'
config = load_config(str(config_path))

print("✅ Configuration loaded from:", config_path)
print("\nKey parameters:")
print(f"  Val ratio: {config['dataset']['val_ratio']}")
print(f"  Test ratio: {config['dataset']['test_ratio']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Frozen epochs: {config['training']['frozen_epochs']}")
print(f"  Fine-tune epochs: {config['training']['fine_tune_epochs']}")

print("\nUnfreezing strategy (Phase 2):")
for backbone, params in config['unfreezing'].items():
    print(f"  {backbone}: {params['phase2_unfreeze_percent']*100:.0f}%")

✅ Configuration loaded from: ../../config/experiment_3.yaml

Key parameters:
  Val ratio: 0.15
  Test ratio: 0.3
  Batch size: 32
  Frozen epochs: 50
  Fine-tune epochs: 100

Unfreezing strategy (Phase 2):


TypeError: string indices must be integers

## Data Loading (Front + Side Views)

In [ ]:
# Load datasets from shared helper
front_base_folder = str(project_root / 'datasets/GEIs_of_rgb_front/GEIs')
side_base_folder = str(project_root / 'datasets/GEIs_of_rgb_side/GEIs')

print(f"Loading GEIs from:\n  Front: {front_base_folder}\n  Side : {side_base_folder}")

dataset, dataset_summary = load_front_side_geis(
    front_base_folder=front_base_folder,
    side_base_folder=side_base_folder,
    seed=42,
    shuffle=True
)

print(f"Merged dataset size: {dataset_summary['total_count']} (front: {dataset_summary['front_count']}, side: {dataset_summary['side_count']})")

if dataset:
    sample_label, sample_img, sample_subject = dataset[0]
    print("Sample tuple structure: (label:str, image:np.ndarray[H,W], subject:str)")
    print(f"Types: {type(sample_label).__name__}, {sample_img.shape}, {type(sample_subject).__name__}")

Merged dataset size: 3142 (front: 1574, side: 1568)
Sample tuple structure: (label:str, image:np.ndarray[H,W], subject:str) -> str (1280, 720) str


## Training Execution (7 Backbones × 5 Runs)

**What's different from Experiment 1:**
- Uses `build_model_for_backbone_v3()` with architecture-specific heads
- **Only 5 runs per backbone** (vs 10 in Experiment 1) for quick validation
- Same 2-phase training strategy (frozen → full unfreeze)
- **3-way subject split (55%/15%/30%)** - validation set for early stopping
- **Higher max epochs** (50+100) but early stopping decides actual epochs

**Expected duration:** ~5-10 minutes per backbone (early stopping reduces training time)

**Total:** 7 backbones × 5 runs = 35 training runs (vs 70 in Experiment 1)

⚠️ **Note:** This cell will take 1-2 hours to complete all training runs!

In [ ]:
# Configure training sweep
BACKBONES_TO_TEST = [
    'efficientnet_b0',
    'efficientnet_b2',
    'efficientnet_b3',
    'resnet50',
    'vgg16',
    'mobilenet_v2',
    'mobilenet_v3_large',
]
N_RUNS = 5

print("="*80)
print("EXPERIMENT 3: Smart Heads Pipeline (delegating to Experiment 4 trainer)")
print("="*80)
print(f"Backbones: {BACKBONES_TO_TEST}")
print(f"Runs per backbone: {N_RUNS}")
print(f"Total training runs: {len(BACKBONES_TO_TEST) * N_RUNS}")
print("="*80)

exp3_run_results = train_experiment_4(
    dataset=dataset,
    backbones=BACKBONES_TO_TEST,
    config_path=str(config_path),
    num_runs=N_RUNS
)

print("\n" + "="*80)
print("✅ TRAINING COMPLETE")
print("="*80)
print("Results saved to: experiments/exer_recog/results/exp_04_regularized/")

Backbones:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.6281 - accuracy: 0.8079
Epoch 1: val_accuracy improved from -inf to 0.75452, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_000_frozen.keras
53/53 [==============================] - 14s 66ms/step - loss: 0.6281 - accuracy: 0.8079 - val_loss: 1.4250 - val_accuracy: 0.7545 - lr: 0.0010
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.0757 - accuracy: 0.9767
Epoch 2: val_accuracy did not improve from 0.75452
53/53 [==============================] - 2s 39ms/step - loss: 0.0757 - accuracy: 0.9767 - val_loss: 1.1320 - val_accuracy: 0.7442 - lr: 0.0010
Epoch 3/50
51/53 [===========================>..] - ETA: 0s - loss: 0.0421 - accuracy: 0.9896
Epoch 3: val_accuracy did not improve from 0.75452
53/53 [==============================] - 2s 39ms/step - loss: 0.0415 - accuracy: 0.9899 - val_loss: 0.8337 - val_accuracy: 0.7364 -

Epoch 1/50
62/62 [==============================] - ETA: 0s - loss: 0.6860 - accuracy: 0.7802
Epoch 1: val_accuracy improved from -inf to 0.65405, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_001_frozen.keras
62/62 [==============================] - 10s 75ms/step - loss: 0.6860 - accuracy: 0.7802 - val_loss: 1.4039 - val_accuracy: 0.6541 - lr: 0.0010
Epoch 2/50
61/62 [============================>.] - ETA: 0s - loss: 0.0821 - accuracy: 0.9790
Epoch 2: val_accuracy improved from 0.65405 to 0.70811, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_001_frozen.keras
62/62 [==============================] - 3s 50ms/step - loss: 0.0832 - accuracy: 0.9785 - val_loss: 1.0818 - val_accuracy: 0.7081 - lr: 0.0010
Epoch 3/50
61/62 [============================>.] - ETA: 0s - loss: 0.0627 - accuracy: 0.9816
Epoch 3: val_accuracy improved f

Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.7150 - accuracy: 0.7780
Epoch 1: val_accuracy improved from -inf to 0.58858, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_002_frozen.keras
53/53 [==============================] - 10s 85ms/step - loss: 0.7150 - accuracy: 0.7780 - val_loss: 1.5793 - val_accuracy: 0.5886 - lr: 0.0010
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.0933 - accuracy: 0.9763
Epoch 2: val_accuracy improved from 0.58858 to 0.76575, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_002_frozen.keras
53/53 [==============================] - 3s 56ms/step - loss: 0.0933 - accuracy: 0.9763 - val_loss: 1.1300 - val_accuracy: 0.7657 - lr: 0.0010
Epoch 3/50
53/53 [==============================] - ETA: 0s - loss: 0.0473 - accuracy: 0.9905
Epoch 3: val_accuracy did not im

Epoch 1/50
57/57 [==============================] - ETA: 0s - loss: 0.6560 - accuracy: 0.8040
Epoch 1: val_accuracy improved from -inf to 0.62445, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_003_frozen.keras
57/57 [==============================] - 12s 95ms/step - loss: 0.6560 - accuracy: 0.8040 - val_loss: 1.5146 - val_accuracy: 0.6245 - lr: 0.0010
Epoch 2/50
55/57 [===========================>..] - ETA: 0s - loss: 0.0996 - accuracy: 0.9705
Epoch 2: val_accuracy improved from 0.62445 to 0.68122, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\efficientnet_b0\run_000_20251119_155602\checkpoints\run_003_frozen.keras
57/57 [==============================] - 3s 54ms/step - loss: 0.1000 - accuracy: 0.9695 - val_loss: 1.1760 - val_accuracy: 0.6812 - lr: 0.0010
Epoch 3/50
57/57 [==============================] - ETA: 0s - loss: 0.0543 - accuracy: 0.9856
Epoch 3: val_accuracy improved f

...

Epoch 1/50
51/53 [===========================>..] - ETA: 0s - loss: 1.6680 - accuracy: 0.4694
Epoch 1: val_accuracy improved from -inf to 0.67587, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_001_frozen.keras
53/53 [==============================] - 7s 67ms/step - loss: 1.6474 - accuracy: 0.4749 - val_loss: 0.8984 - val_accuracy: 0.6759 - lr: 0.0010
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.5206 - accuracy: 0.8479
Epoch 2: val_accuracy improved from 0.67587 to 0.76611, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_001_frozen.keras
53/53 [==============================] - 2s 44ms/step - loss: 0.5206 - accuracy: 0.8479 - val_loss: 0.6865 - val_accuracy: 0.7661 - lr: 0.0010
Epoch 3/50
53/53 [==============================] - ETA: 0s - loss: 0.2589 - accuracy: 0.9272
Epoch 3: val_accuracy did n

Epoch 1/50
56/56 [==============================] - ETA: 0s - loss: 1.5976 - accuracy: 0.4847
Epoch 1: val_accuracy improved from -inf to 0.30556, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_002_frozen.keras
56/56 [==============================] - 6s 61ms/step - loss: 1.5976 - accuracy: 0.4847 - val_loss: 2.3896 - val_accuracy: 0.3056 - lr: 0.0010
Epoch 2/50
56/56 [==============================] - ETA: 0s - loss: 0.4724 - accuracy: 0.8667
Epoch 2: val_accuracy improved from 0.30556 to 0.39352, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_002_frozen.keras
56/56 [==============================] - 2s 40ms/step - loss: 0.4724 - accuracy: 0.8667 - val_loss: 1.7414 - val_accuracy: 0.3935 - lr: 0.0010
Epoch 3/50
53/56 [===========================>..] - ETA: 0s - loss: 0.2205 - accuracy: 0.9304
Epoch 3: val_accuracy impro

Epoch 1/50
52/53 [============================>.] - ETA: 0s - loss: 1.7322 - accuracy: 0.4615
Epoch 1: val_accuracy improved from -inf to 0.44969, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_003_frozen.keras
53/53 [==============================] - 6s 65ms/step - loss: 1.7313 - accuracy: 0.4619 - val_loss: 1.4416 - val_accuracy: 0.4497 - lr: 0.0010
Epoch 2/50
51/53 [===========================>..] - ETA: 0s - loss: 0.5666 - accuracy: 0.8241
Epoch 2: val_accuracy improved from 0.44969 to 0.60986, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_003_frozen.keras
53/53 [==============================] - 3s 47ms/step - loss: 0.5667 - accuracy: 0.8246 - val_loss: 1.1589 - val_accuracy: 0.6099 - lr: 0.0010
Epoch 3/50
52/53 [============================>.] - ETA: 0s - loss: 0.2599 - accuracy: 0.9243
Epoch 3: val_accuracy impro

Epoch 1/50
52/53 [============================>.] - ETA: 0s - loss: 1.6127 - accuracy: 0.4982
Epoch 1: val_accuracy improved from -inf to 0.64392, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_004_frozen.keras
53/53 [==============================] - 6s 65ms/step - loss: 1.6024 - accuracy: 0.5018 - val_loss: 1.1103 - val_accuracy: 0.6439 - lr: 0.0010
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.4513 - accuracy: 0.8609
Epoch 2: val_accuracy improved from 0.64392 to 0.66098, saving model to ../../experiments/exer_recog/results/exp_03_smart_heads\mobilenet_v3_large\run_000_20251119_175545\checkpoints\run_004_frozen.keras
53/53 [==============================] - 2s 44ms/step - loss: 0.4513 - accuracy: 0.8609 - val_loss: 0.9560 - val_accuracy: 0.6610 - lr: 0.0010
Epoch 3/50
52/53 [============================>.] - ETA: 0s - loss: 0.2375 - accuracy: 0.9297
Epoch 3: val_accuracy did n

Backbones: 100%|██████████| 7/7 [2:07:53<00:00, 1096.27s/it]


## Comprehensive Analysis

In [13]:
print("="*80)
print("COMPREHENSIVE BACKBONE COMPARISON (Experiment 3)")
print("="*80)

# Step 1: Count model parameters
backbones_to_analyze = [
    'efficientnet_b0',
    'efficientnet_b2',
    'efficientnet_b3',
    'resnet50',
    'vgg16',
    'mobilenet_v2',
    'mobilenet_v3_large',
]

print("\nStep 1: Counting model parameters...")
params_df = get_all_model_parameters(backbones_to_analyze, img_size=224, num_classes=15)

print("\nModel Parameter Comparison:")
print("="*80)
print(params_df.to_string(index=False))
print("="*80)

# Step 2: Load training results
print("\nStep 2: Loading training results...")
backbone_results = load_backbone_results_with_config(results_base_dir='../../experiments/exer_recog/results/exp_03_smart_heads')

# Step 3: Generate comparisons
print("\nStep 3: Generating comprehensive comparison...")
os.makedirs('experiments/exer_recog/results/exp_03_smart_heads/comparisons', exist_ok=True)
comparison_csv = create_comprehensive_comparison(
    all_backbone_results=backbone_results,
    model_params_df=params_df,
    output_dir='D:/Graduation Project/ai-virtual-coach/experiments/exer_recog/results/exp_03_smart_heads/comparisons'
)

# Step 4: Statistical analysis
print("\nStep 4: Performing statistical analysis...")
stats_txt = generate_statistical_comparison(
    all_backbone_results=backbone_results,
    output_dir='D:/Graduation Project/ai-virtual-coach/experiments/exer_recog/results/exp_03_smart_heads/comparisons'
)

print("\n✓ Analysis complete! Results saved to: experiments/exer_recog/results/exp_03_smart_heads/comparisons/")

COMPREHENSIVE BACKBONE COMPARISON (Experiment 3)

Step 1: Counting model parameters...


Counting parameters: 100%|██████████| 7/7 [00:14<00:00,  2.12s/it]
No run folders found in ../../experiments/exer_recog/results/exp_03_smart_heads\exp_03_smart_heads



Model Parameter Comparison:
          backbone  total_params  trainable_params  non_trainable_params  num_layers  params_millions
      mobilenet_v2       2589775            331791               2257984           6             2.59
mobilenet_v3_large       3246223            249871               2996352           6             3.25
   efficientnet_b0       6251103            331791               5919312           6             6.25
   efficientnet_b2       9133933            364559               8769374           6             9.13
   efficientnet_b3      13327949            397327              12930622           6            13.33
             vgg16      14849871            135183              14714688           6            14.85
          resnet50      24116111            528399              23587712           6            24.12

Step 2: Loading training results...

Step 3: Generating comprehensive comparison...

Step 4: Performing statistical analysis...

✓ Analysis complete! Resu

## Compare with Experiment 1 Baseline

This cell loads results from both Experiment 1 (baseline) and Experiment 3 (smart heads) to compare performance.

In [ ]:
print("="*80)
print("EXPERIMENT 1 (BASELINE) vs EXPERIMENT 3 (SMART HEADS)")
print("="*80)

# Load Experiment 1 results
exp1_results = load_backbone_results_with_config(results_base_dir='experiments/exer_recog/results/exp_01_baseline')

# Load Experiment 3 results
exp3_results = load_backbone_results_with_config(results_base_dir='experiments/exer_recog/results/exp_03_smart_heads')

# Create comparison dataframe
comparison_data = []

for backbone in BACKBONES_TO_TEST:
    if backbone in exp1_results and backbone in exp3_results:
        exp1_accs = [r['test_acc'] for r in exp1_results[backbone]]
        exp3_accs = [r['test_acc'] for r in exp3_results[backbone]]
        
        comparison_data.append({
            'Backbone': backbone,
            'Exp1_Mean': np.mean(exp1_accs),
            'Exp1_Std': np.std(exp1_accs),
            'Exp1_Best': np.max(exp1_accs),
            'Exp3_Mean': np.mean(exp3_accs),
            'Exp3_Std': np.std(exp3_accs),
            'Exp3_Best': np.max(exp3_accs),
            'Mean_Improvement': np.mean(exp3_accs) - np.mean(exp1_accs),
            'Best_Improvement': np.max(exp3_accs) - np.max(exp1_accs)
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Exp3_Mean', ascending=False)

print("\nDetailed Comparison:")
print(comparison_df.to_string(index=False))

# Summary statistics
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Best Exp1 Mean: {comparison_df['Exp1_Mean'].max():.4f} ({comparison_df.loc[comparison_df['Exp1_Mean'].idxmax(), 'Backbone']})")
print(f"Best Exp3 Mean: {comparison_df['Exp3_Mean'].max():.4f} ({comparison_df.loc[comparison_df['Exp3_Mean'].idxmax(), 'Backbone']})")
print(f"\nAverage improvement: {comparison_df['Mean_Improvement'].mean():.4f}")
print(f"Backbones improved: {(comparison_df['Mean_Improvement'] > 0).sum()}/{len(comparison_df)}")

# Check success criteria
best_exp3_mean = comparison_df['Exp3_Mean'].max()
best_exp3_std = comparison_df.loc[comparison_df['Exp3_Mean'].idxmax(), 'Exp3_Std']

print("\n" + "="*80)
print("SUCCESS CRITERIA CHECK")
print("="*80)
print(f"Best Exp3 mean: {best_exp3_mean:.4f} (std={best_exp3_std:.4f})")

if best_exp3_mean > 0.88 and best_exp3_std < 0.02:
    print("✅ EXCELLENT SUCCESS: Mean > 88% and std < 2%")
    print("   → Ready to scale to 30 runs!")
elif best_exp3_mean > 0.86 and best_exp3_std < 0.025:
    print("✅ GOOD SUCCESS: Mean > 86% and std < 2.5%")
    print("   → Consider scaling to 30 runs for top performers")
elif best_exp3_mean > 0.84:
    print("✅ MINIMUM SUCCESS: Mean > 84% (beats baseline 83.25%)")
    print("   → Architecture-specific heads show promise, but may need refinement")
else:
    print("❌ BELOW BASELINE: Mean ≤ 84%")
    print("   → Architecture-specific heads did not improve performance")
    print("   → Consider reverting to universal heads or refining head designs")

# Save comparison
comparison_df.to_csv('experiments/exer_recog/results/exp_03_smart_heads/exp1_vs_exp3_comparison.csv', index=False)
print("\nComparison saved to: experiments/exer_recog/results/exp_03_smart_heads/exp1_vs_exp3_comparison.csv")

---

## Experiment 3 Complete! 🎉

### Results Summary
All results saved to: `experiments/exer_recog/results/exp_03_smart_heads/`

**Generated files:**
- `backbone_comparison.csv` - Quick comparison table
- `exp1_vs_exp3_comparison.csv` - Direct comparison with baseline
- `comparisons/comprehensive_comparison.csv` - Detailed metrics
- `comparisons/statistical_analysis.txt` - Statistical tests
- Individual backbone folders with run-specific results

### Key Questions to Answer

**1. Did architecture-specific heads improve performance?**
- Compare mean test accuracy: Exp3 vs Exp1 baseline
- Check if any backbone beats baseline 83.25%

**2. Which backbone family benefits most?**
- EfficientNet with swish activation?
- ResNet50 with larger head?
- MobileNet with compact head?

**3. Is consistency improved?**
- Compare std deviation: Exp3 vs Exp1
- Lower std = more reliable deployment performance

### Next Steps

**If Excellent Success (mean > 88%, std < 2%):**
- 🚀 Scale to 30 runs for best backbone
- Target: Find single model with >90% test accuracy
- Deploy best performer for production

**If Good Success (mean > 86%, std < 2.5%):**
- 📈 Scale to 30 runs for top 2-3 backbones
- Compare statistical significance
- Consider ensemble of top performers

**If Minimum Success (mean > 84%):**
- 🔍 Analyze which head designs worked best
- Refine underperforming heads
- Try alternative dropout rates or layer sizes

**If Below Baseline (mean ≤ 84%):**
- ❌ Revert to universal heads (Experiment 1 approach)
- Focus on other improvements: augmentation, ensemble methods
- Consider that universal heads may be optimal for this task

### Design Philosophy Validated?

This experiment tests the hypothesis that **architecture-specific heads outperform universal heads** when properly designed:
- ✅ GlobalAveragePooling2D for all backbones
- ✅ 2-layer heads (balanced capacity)
- ✅ Activation matching (swish for EfficientNet, relu for others)
- ✅ Progressive dropout
- ✅ Size scaling to backbone complexity

Results will confirm or refute this hypothesis and guide future experiments!